<a href="https://colab.research.google.com/github/OSAMA-2003/Ai-project/blob/main/Ai_Systems_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install sklearn-genetic
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.optimize import differential_evolution
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import check_random_state
# from genetic_selection import GeneticSelectionCV

In [3]:
# Step 1: Data Preparation
# Load the dataset
data = pd.read_csv('/content/Heart_Dataset.csv')

X = data.iloc[:,2:]
Y= data.iloc[:,1:2]
X.replace(np.nan,inplace=True)
# x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.30, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.05, random_state = 50)

In [ ]:

# 1-Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(x_train, y_train)

# Predict
y_pred_dt = dt_classifier.predict(x_test)

# Accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f'Decision Tree Accuracy: {round((accuracy_dt*100), 2)}%')

In [ ]:

# 2- Neural Networks MLP

from sklearn.neural_network import MLPClassifier

# Multi-layer Perceptron (MLP) Classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)
mlp_classifier.fit(x_train, y_train)

# Predict
y_pred_mlp = mlp_classifier.predict(x_test)

# Accuracy
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
# print("MLP Accuracy:", accuracy_mlp)
print(f'MLP Accuracy: {round((accuracy_mlp*100), 2)}%')

In [ ]:


# !pip install sklearn-genetic

# 3. Genetic Algorithms for Feature Selection
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Define the classifier
classifier = RandomForestClassifier(n_estimators=10, random_state=500)

# Define the genetic selection
selector = GeneticSelectionCV(classifier,cv=5,verbose=1,scoring="accuracy",crossover_proba=0.5, mutation_proba=0.2,n_generations=40, crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05, tournament_size=3, n_gen_no_change=10,caching=True, n_jobs=-1)

# Perform genetic feature selection
selector.fit(X_train_scaled, y_train)

# Transform the training and testing sets to selected features
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

# Train the classifier on the selected features
classifier.fit(X_train_selected, y_train)

# Predict the labels of the test set
y_pred = classifier.predict(X_test_selected)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f'Genetic Algorithms Accuracy: {round((accuracy*100), 2)}%')



In [ ]:
# 4-Differential Evolution Algorithm for Hyperparameter Optimization (Neural Network)
# Define the fitness function
def fitness_function(params, x_train, y_train, x_test, y_test):
    n_estimators = int(params[0])
    max_depth = int(params[1])
    min_samples_split = int(params[2])
    min_samples_leaf = int(params[3])
    # Create and train the random forest classifier
    classifier = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,random_state=42)
    classifier.fit(x_train, y_train)
    # Predict the labels of the test set
    y_pred = classifier.predict(x_test)
    # Calculate and return the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy for optimization
# Define the bounds for the hyperparameters
bounds = [(10, 200),  # Number of estimators
          (1, 20),  # Maximum depth
          (2, 20),  # Minimum samples split
          (1, 20)]  # Minimum samples leaf

# Perform differential evolution for hyperparameter optimization
result = differential_evolution( fitness_function, bounds=bounds,args=(x_train, y_train, x_test, y_test),strategy='best1bin', popsize=10,  # Population size
                                mutation=(0.5, 1),  recombination=0.7, maxiter=20, tol=0.01, seed=42)
# Get the best hyperparameters
n_estimators, max_depth, min_samples_split, min_samples_leaf = result.x

# Create and train the random forest classifier with the best hyperparameters
classifier = RandomForestClassifier(n_estimators=int(n_estimators),max_depth=int(max_depth),min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf),
 random_state=42)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Differential Evolution Algorithm Accuracy: {round((accuracy*100), 2)}%')

In [ ]:
# 5-Principal Component Analysis (PCA)
from sklearn.decomposition import PCA

# PCA
pca = PCA(n_components=0.95, random_state=42)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

# MLP Classifier with PCA
mlp_classifier_pca = MLPClassifier(max_iter=1000, activation='relu', solver='adam', random_state=42)
mlp_classifier_pca.fit(x_train_pca, y_train)

# Predict
y_pred_mlp_pca = mlp_classifier_pca.predict(x_test_pca)

# Accuracy
accuracy_mlp_pca = accuracy_score(y_test, y_pred_mlp_pca)
print(f'MLP with PCA Accuracy: {round((accuracy_mlp_pca*100), 2)}%')
